In [1]:
# Using the pcap https://mrncciew.files.wordpress.com/2014/08/wpa2-psk-final.zip for testing

from binascii import a2b_hex, b2a_hex, a2b_qp
from pbkdf2 import PBKDF2
import hmac
from hashlib import sha1
import struct
from Crypto.Cipher import AES

In [2]:
def PRF512(key,A,B):
    blen = 64
    R    = ''
    for i in range(0,4):
        hmacsha1 = hmac.new(key,A+B+chr(i),sha1)
        R = R+hmacsha1.digest()
    return R[:blen]  

In [3]:
def frame_type(packet):
    header_two_bytes = struct.unpack("h", (packet[0:2]))[0]
    fc_type = bin(header_two_bytes)[-8:][4:6]
    if fc_type == "10":
        return "data"
    else:
        return None

In [4]:
def compute_pairwise_master_key(preshared_key, ssid):
    return PBKDF2(preshared_key, ssid, 4096).read(32)

def compute_message_integrity_check(pairwise_transient_key,data):
    return hmac.new(pairwise_transient_key[0:16],data,sha1).digest()[0:16]

def compute_pairwise_transient_key(pairwise_master_key, A, B):
    return PRF512(pairwise_master_key, A, B)

In [5]:
ssid = "TEST1"
preshared_key = "Cisco123Cisco123"

# From message 2 in handshake QoS data for 802.11, packet 95 in example pcap
message_2_data = "88012c0064a0e7af474e001bd458e61a64a0e7af474e10000600aaaa03000000888e0203007502010a00100000000000000000605e85a79cfafdb0eaa050683f97be1b66def7bc652057316871c273c5ae477f00000000000000000000000000000000000000000000000000000000000000009189cdf188548e73cd37d57852660588001630140100000fac040100000fac040100000fac022800"
message_2_data = a2b_hex(message_2_data)

message_intgrity_code = message_2_data[115:131]
data = message_2_data[34:115] + "\x00"*16 + message_2_data[131:]

In [6]:
# authenticator nonce found in message 1 of handshake, packet 93 in example
a_nonce  = a2b_hex("126ace64c1a644d27b84e039263b633bc374e3299d7d45e1c42544054805bfe5")

# supplicant nonce found in message 2 of handshake, packet 95 in example
s_nonce  = a2b_hex("605e85a79cfafdb0eaa050683f97be1b66def7bc652057316871c273c5ae477f")

mac_access_point = a2b_hex("64a0e7af474e")
mac_client   = a2b_hex("001bd458e61a")

A = "Pairwise key expansion" + '\x00'
B = min(mac_access_point,mac_client)+max(mac_access_point,mac_client)+min(a_nonce,s_nonce)+max(a_nonce,s_nonce)

In [7]:
pairwise_master_key = compute_pairwise_master_key(preshared_key, ssid)
pairwise_transient_key = compute_pairwise_transient_key(pairwise_master_key, A, B)
mic = compute_message_integrity_check(pairwise_transient_key,data)

In [8]:
# See the mic for packet 95
print b2a_hex(mic) == "9189cdf188548e73cd37d57852660588"

True


In [9]:
key_confirmation_key = pairwise_transient_key[0:16]
key_encryption_key = pairwise_transient_key[16:16*2]
temporal_key = pairwise_transient_key[16 * 2:(16 * 2) + 16]
mic_authenticator_tx = pairwise_transient_key[16 * 3:(16 * 3) + 8]
mic_authenticator_rx =  pairwise_transient_key[(16 * 3) + 8:(16 * 3) + 8 + 8]

In [45]:
packet_103_encrypted_total_packet = "88512c0064a0e7af474e001bd458e61a64a0e7af474e1000000001000020000000003d833a329d1d0ccc0372742ab1031986151713a0c160a6035752c214ce42bebdaefad728edf8a728e013fb3939046e0823753aa4339f953c197d49e10a1e1aa2d19175bcf61e81867af5c8024a3d29ebce166e4401ea6994"
packet_103_encrypted_total_packet = a2b_hex(packet_103_encrypted_total_packet)
packet_103_encrypted_data = packet_103_encrypted_total_packet[34:34+84]

In [46]:
ccmp_header = packet_103_encrypted_total_packet[26:26 + 8]
ieee80211_header = packet_103_encrypted_total_packet[0:26]
source_address = packet_103_encrypted_total_packet[10:16]

In [47]:
PN5 = ccmp_header[7]
PN4 =  ccmp_header[6]
PN3 =  ccmp_header[5]
PN2 =  ccmp_header[4]
PN1 =  ccmp_header[1]
PN0 =  ccmp_header[0]

last_part_of_nonce = PN5 + PN4 + PN3 + PN2 + PN1 + PN0

flag = a2b_hex('01')
qos_priorty = a2b_hex('00')

In [48]:
nonce_ = qos_priorty + source_address + last_part_of_nonce
IV = flag + nonce_

In [49]:
class WPA2Counter(object):
  def __init__(self, secret):
    self.secret = secret
    self.current = 1
  def counter(self):
    count = a2b_hex(struct.pack('>h', self.current).encode('hex'))
    i = self.secret + count
    self.current += 1
    return i

In [51]:
counter = WPA2Counter(IV)
crypto = AES.new(temporal_key, AES.MODE_CTR, counter=counter.counter)
test = packet_103_encrypted_data[0:-8]
crypto.decrypt(test).encode('hex')

'aaaa030040960000004c300064a0e7af474e001bd458e61a009b00300040960064a0e7af474f002400064d79486f6d6500000000000000000000000000000000000000000000000000000416'